# Completion Rate (CR)

## Carga de datos

In [ ]:
# --------------------------------------------------
# PASO 1: IMPORTAR LIBRERÍAS
# --------------------------------------------------
import pandas as pd
import numpy as np
import statsmodels.api as sm

# --------------------------------------------------
# PASO 2: CARGAR DATOS
# --------------------------------------------------
# Ajusta la ruta según tu proyecto:
df = pd.read_csv("../../data/processed/navegacion_clientes_experimento_limpio.csv")

In [ ]:
# --------------------------------------------------
# PASO 3: DEFINIR "COMPLETÓ SESIÓN"
# --------------------------------------------------
# Suponiendo que llegar a 'confirm' implica completar el flujo:
df["completed"] = df["process_step"] == "confirm"

# --------------------------------------------------
# PASO 4: AGRUPAR POR (variation, visit_id)
# --------------------------------------------------
# Cada visit_id es una sesión. Tomamos la "máxima" de 'completed'
# para saber si en esa sesión hubo al menos un 'confirm'.
df_sessions = (df.groupby(["variation", "visit_id"], as_index=False).agg({"completed": "max"}))

# --------------------------------------------------
# PASO 5: CALCULAR COMPLETION RATE POR GRUPO
# --------------------------------------------------
cr_data = df_sessions.groupby("variation")["completed"].agg(["mean", "sum", "count"])
print("=== COMPLETION RATE POR GRUPO (variation) ===")
print(cr_data)
print("\n- mean  = Tasa de finalización (Completion Rate)")
print("- sum   = Nº de sesiones completadas")
print("- count = Nº total de sesiones\n")

# --------------------------------------------------
# PASO 6: TEST DE HIPÓTESIS (COMPARACIÓN DE PROPORCIONES)
# --------------------------------------------------
# H0: CR (Test) = CR (Control)
# H1: CR (Test) != CR (Control)   (test bilateral, por ejemplo)

test_sum = cr_data.loc["Test",   "sum"]   # Nº de sesiones completadas en Test
test_cnt = cr_data.loc["Test",   "count"] # Total de sesiones en Test
ctrl_sum = cr_data.loc["Control","sum"]   # Nº de sesiones completadas en Control
ctrl_cnt = cr_data.loc["Control","count"] # Total de sesiones en Control

count = np.array([test_sum, ctrl_sum])  # sesiones completadas en cada grupo
nobs  = np.array([test_cnt, ctrl_cnt])  # total de sesiones en cada grupo

stat, p_value = sm.stats.proportions_ztest(count, nobs, alternative='two-sided')
print("=== PRUEBA DE HIPÓTESIS PARA COMPLETION RATE ===")
print(f"Estadístico z: {stat:.4f}")
print(f"p-value     : {p_value:.6f}")

# --------------------------------------------------
# PASO 7: INTERPRETAR RESULTADOS
# --------------------------------------------------
alpha = 0.05  # nivel de significancia (5%)

if p_value < alpha:
    print(f"\nConclusión: Rechazamos H0 (p < {alpha}).")
    print("Hay diferencia estadísticamente significativa en la Completion Rate entre Test y Control.")
else:
    print(f"\nConclusión: No rechazamos H0 (p >= {alpha}).")
    print("No se detecta diferencia estadísticamente significativa en la Completion Rate entre Test y Control.")

# --------------------------------------------------
# (Opcional) PASO 8: IMPRIMIR DATOS RESUMIDOS
# --------------------------------------------------
cr_test    = cr_data.loc["Test",   "mean"]  # CR del grupo Test
cr_control = cr_data.loc["Control","mean"]  # CR del grupo Control

print(f"\n=== Resumen Final ===")
print(f"Completion Rate (Test)    : {cr_test:.3%}")
print(f"Completion Rate (Control) : {cr_control:.3%}")
print(f"p-value                   : {p_value:.6f}")


=== COMPLETION RATE POR GRUPO (variation) ===
               mean    sum  count
variation                        
Control    0.498439  15006  30106
Test       0.580451  20173  34754
Unknown    0.544714  14113  25909

- mean  = Tasa de finalización (Completion Rate)
- sum   = Nº de sesiones completadas
- count = Nº total de sesiones

=== PRUEBA DE HIPÓTESIS PARA COMPLETION RATE ===
Estadístico z: 20.9081
p-value     : 0.000000

Conclusión: Rechazamos H0 (p < 0.05).
Hay diferencia estadísticamente significativa en la Completion Rate entre Test y Control.

=== Resumen Final ===
Completion Rate (Test)    : 58.045%
Completion Rate (Control) : 49.844%
p-value                   : 0.000000
